# End-to-End Demonstration: Google Classroom

*Goal*: run the Google Classroom Extractor and then upload the results into an `LMS` database.

## Software Requirements

1. Be sure to install Python 3.9; if you have multiple versions, make sure that the `python` command runs version 3.9.x. You can confirm your version by running `python --version` at a command prompt.
1. Microsoft SQL Server 2017 or 2019, in Windows or Linux.

## Getting Started

1. Confirm you have [poetry](https://python-poetry.org) installed (`poetry --version`).
1. Follow the [Google Classroom setup instructions](../google-classroom/README.md) in order to create a `service-account.json` file.
1. Follow the [notebook instructions](README.md) to install dependencies used by this notebook.
1. Create an `LMS` database in SQL Server.

In [ ]:
# Load some utilities
from IPython.display import display, Markdown

# Setup logging
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

## Prepare Input Data

Update the variables in the next block as needed.

In [ ]:
CLASSROOM_ACCOUNT = "admin@ibamonitoring.org"
START_DATE = "2020-08-17"
END_DATE = "2021-05-23"
LOG_LEVEL = "INFO"
OUTPUT_DIRECTORY = "gc-data"
SYNC_DATABASE_DIRECTORY=OUTPUT_DIRECTORY
DB_ENGINE = "mssql"
DB_SERVER = "localhost"
DB_NAME = "LMS"
DB_PORT = 1433
EXTRACT_ASSIGNMENTS = True
EXTRACT_ACTIVITIES = False
EXTRACT_ATTENDANCE = False
EXTRACT_GRADES = False

## Run the Google Classroom Extractor

In [ ]:
from edfi_google_classroom_extractor.helpers.arg_parser import MainArguments as gc_args
from edfi_google_classroom_extractor import facade

arguments = gc_args(
    classroom_account=CLASSROOM_ACCOUNT,
    log_level=LOG_LEVEL,
    output_directory=OUTPUT_DIRECTORY,
    usage_start_date=START_DATE,
    usage_end_date=END_DATE,
    sync_database_directory=SYNC_DATABASE_DIRECTORY,
    extract_assignments=EXTRACT_ASSIGNMENTS,
    extract_activities=EXTRACT_ACTIVITIES,
    extract_attendance=EXTRACT_ATTENDANCE,
    extract_grades=EXTRACT_GRADES,
)

facade.run(arguments)

## Run the Learning Management System Data Store Loader (LMS-DS-Loader)

The default setup below uses Windows integrated security. For username/password security, please review the commented-out code.

In [ ]:
from edfi_lms_ds_loader.helpers.argparser import MainArguments as lms_args
from edfi_lms_ds_loader import loader_facade

arguments = lms_args(
    OUTPUT_DIRECTORY,
    DB_ENGINE,
    LOG_LEVEL
)
arguments.set_connection_string_using_integrated_security(
    DB_SERVER,
    DB_PORT,
    DB_NAME,
)
# For password auth, comment out the line above and uncomment this one:
# arguments.set_connection_string(
#     DB_SERVER,
#     DB_PORT,
#     DB_NAME,
#     USERNAME,
#     PASSWORD,
# )

loader_facade.run_loader(arguments)